In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table,con,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursorUAT.execute(sql_query)
            connUAT.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            con.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

sql_check = '''
            select * from staging.stg_tbdm_movies_json
            '''
sql_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
sql_json.sort_values(by='id',inplace=True)
sql_json

,id,request_json
36,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
0,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
1,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
2,6,"{""adult"": false, ""backdrop_path"": ""/iUUpKunmBN..."
3,11,"{""adult"": false, ""backdrop_path"": ""/2w4xG178Rp..."
...,...,...
70395,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
86407,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
86408,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
86409,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


In [2]:
fall_m = sql_json[sql_json['request_json']=='{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}']

### sử dụng:
- belongs_to_collection: dùng id để lọc những film liên quan lẫn nhau
- Dữ liệu có 126 movieid ko có mã tbdmid
- 1071 movieid có mã tbdmid những ko có dữ liệu trên server 
    - code: tbdm movies_info[movies_info['tmdbId'].isin(list(fall_m['id']))]


In [3]:
tmdb_data = json.loads(sql_json[sql_json['id']==11]['request_json'].values[0])
# tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [4]:
tmdb_data = json.loads(sql_json[sql_json['id']==1891]['request_json'].values[0])
# tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [5]:
# [tmdb_data['production_countries'][0]['name'],tmdb_data['spoken_languages'][0]['english_name']
#  ,tmdb_data['production_companies'][0]['name']
#  ,tmdb_data['overview'],tmdb_data['status'],tmdb_data['release_date']
# ]

In [6]:
# movies_links[movies_links['tmdbId'].isnull()]

In [7]:
movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links.sort_values(by='tmdbId',inplace=True)
movies_info = links.merge(movies, on='movieId', how='left')
del movies_info['genres']
# movies.sort_values(by='tmdbId',inplace=True)
movies_info


,movieId,imdbId,tmdbId,title
0,4470,94675,2.0,Ariel (1988)
1,61724,92149,3.0,Shadows in Paradise (Varjoja paratiisissa) (1986)
2,18,113101,5.0,Four Rooms (1995)
3,479,107286,6.0,Judgment Night (1993)
4,260,76759,11.0,Star Wars: Episode IV - A New Hope (1977)
...,...,...,...,...
86532,222211,7688638,NaN,Good Girls Get High (2018)
86533,224398,5868094,NaN,She Loves Me (2016)
86534,234301,5714216,NaN,Black Butler: Book of Murder (2014)
86535,237231,9372822,NaN,Tenderness (2018)


In [8]:
movies_info.isnull().sum()

movieId      0
imdbId       0
tmdbId     126
title        0
dtype: int64

In [9]:
SQL_push(
         sql_table_name='staging.stg_grouplen_movieinfo',
         python_table=movies_info,
         con=connUAT,
         cursor=cursorUAT,
         inplace= True
        )

Inserting rows: 100%|██████████| 86537/86537 [00:07<00:00, 11200.89it/s]

PUSH DATA: DONE


In [8]:
# DONE
movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
movies1 = movies[['movieId','genres']]
max_col = [len(i) for i in list(movies1['genres'].str.split('|'))]
movies_genres = pd.DataFrame()

for i in range(max(max_col)):
    data = movies.copy()
    data['genres'] = data['genres'].str.split('|',expand=True)[i]
    data = data[data['genres'].isnull()==False]
    movies_genres = pd.concat([movies_genres,data])

movies_genres.sort_values(by='movieId',ascending=True,inplace=True)
del movies_genres['title']
movies_genres

,movieId,genres
0,1,Adventure
0,1,Fantasy
0,1,Animation
0,1,Comedy
0,1,Children
...,...,...
86533,288971,Horror
86534,288975,Documentary
86535,288977,Thriller
86535,288977,Crime


In [9]:
movies_genres.isnull().sum()

movieId    0
genres     0
dtype: int64

In [10]:
SQL_push(
         sql_table_name='staging.stg_grouplen_moviesgenres',
         python_table=movies_genres,
         con=connUAT,
         cursor=cursorUAT,
         inplace= True
        )

Inserting rows: 100%|██████████| 152288/152288 [00:13<00:00, 11712.52it/s]

PUSH DATA: DONE


In [11]:
# DONE
moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags['date_time'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
# moives_tags['date'] = pd.to_datetime(moives_tags['date_time'],format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
del moives_tags['timestamp']
# tags.sort_values(by='tmdbId',inplace=True)
moives_tags

,userId,movieId,tag,date_time
0,10,260,good vs evil,2015-05-03 15:22:38
1,10,260,harrison ford,2015-05-03 15:21:45
2,10,260,sci-fi,2015-05-03 15:22:18
3,14,1221,al pacino,2011-07-25 13:32:36
4,14,1221,mafia,2011-07-25 13:32:26
...,...,...,...,...
2328310,330923,176599,politically correct,2017-10-09 11:11:31
2328311,330933,3317,coming of age,2012-10-26 19:23:04
2328312,330933,3317,sexuality,2012-10-26 19:23:09
2328313,330947,5782,not luc besson,2006-07-28 18:21:42


In [12]:
moives_tags.isnull().sum()

userId        0
movieId       0
tag          27
date_time     0
dtype: int64

In [13]:
SQL_push(
            sql_table_name= 'staging.stg_grouplen_moviestags',
            python_table= moives_tags,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

Inserting rows: 100%|██████████| 2328315/2328315 [04:03<00:00, 9568.43it/s] 


PUSH DATA: DONE


In [14]:
# DONE
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['date_time'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings

,userId,movieId,rating,date_time
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39
...,...,...,...,...
33832157,330975,8340,2.0,2004-08-04 01:34:16
33832158,330975,8493,2.5,2004-08-04 02:15:09
33832159,330975,8622,4.0,2004-08-04 01:09:37
33832160,330975,8665,3.0,2004-08-04 01:09:25


In [15]:
movies_ratings.isnull().sum()

userId       0
movieId      0
rating       0
date_time    0
dtype: int64

In [17]:
SQL_push(
            sql_table_name= 'staging.stg_grouplen_moviesratings',
            python_table= movies_ratings,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

Inserting rows: 100%|██████████| 33832162/33832162 [1:09:39<00:00, 8095.11it/s] 


PUSH DATA: DONE


In [18]:
# DONE
genome_s = pd.read_csv(os.path.join(pathfolder,'genome-scores.csv'))
genome_s

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300
...,...,...,...
18472123,288167,1124,0.09875
18472124,288167,1125,0.02950
18472125,288167,1126,0.02275
18472126,288167,1127,0.11225


In [19]:
genome_s.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

In [20]:
SQL_push(
            sql_table_name= 'staging.stg_genome_scores',
            python_table= genome_s,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

Inserting rows: 100%|██████████| 18472128/18472128 [31:58<00:00, 9626.37it/s] 


PUSH DATA: DONE


In [21]:
# DONE
genome_t = pd.read_csv(os.path.join(pathfolder,'genome-tags.csv'))
genome_t

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [22]:
genome_t.isnull().sum()

tagId    0
tag      0
dtype: int64

In [23]:
SQL_push(
            sql_table_name= 'staging.stg_genome_tags',
            python_table= genome_t,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

Inserting rows: 100%|██████████| 1128/1128 [00:00<00:00, 7412.66it/s]

PUSH DATA: DONE
